In [0]:
# Use seaborn for pairplot
!pip install -q seaborn
!pip install tensorflow==2.0.0
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs
!pip install h5py pyyaml 

     |████████████████████████████████| 86.3MB 58kB/s 
     |████████████████████████████████| 450kB 47.7MB/s 
     |████████████████████████████████| 3.8MB 29.0MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib


import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
import warnings
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")
import pickle

In [0]:
# create and save all the models
airlines = pd.read_csv('drive/My Drive/airlines.csv')
carriers = list(airlines['IATA_CODE'])
print(carriers)

['UA', 'AA', 'US', 'F9', 'B6', 'OO', 'AS', 'NK', 'WN', 'DL', 'EV', 'HA', 'MQ', 'VX']


In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


In [0]:
def build_model(train_ds):
  model = keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[len(train_ds.keys())]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
def build_model(train_ds):
  model = keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=[len(train_ds.keys())]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'accuracy'])
  return model

In [0]:
for carrier in carriers:
  # create a model and save it for each carrier
  df = pd.read_csv('drive/My Drive/carriers/carrier' + str(carrier) + 'data.csv')
  df.drop(['Unnamed: 0'], axis=1, inplace=True)

  # encode the origin 
  encoder = LabelEncoder()
  encoder.fit(df['ORIGIN_AIRPORT'])
  encoded_data_map = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
  df['ORIGIN_AIRPORT'] = encoder.fit_transform(df['ORIGIN_AIRPORT'])

  # create the train and test dataset
  train_dataset = df.sample(frac=0.8,random_state=0)
  test_dataset = df.drop(train_dataset.index)

  # getting the stats
  train_stats = train_dataset.describe()
  train_stats.pop("ARRIVAL_DELAY")
  train_stats = train_stats.transpose()

  # defining the train and test labels
  train_labels = train_dataset.pop('ARRIVAL_DELAY')
  test_labels = test_dataset.pop('ARRIVAL_DELAY')

  # normalize the data
  normed_train_data = norm(train_dataset)
  normed_test_data = norm(test_dataset)

  # define the model
  model = build_model(train_dataset)


  # train the model
  EPOCHS = 100
  # The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

  early_history = model.fit(normed_train_data, train_labels, 
                      epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                      callbacks=[early_stop, tfdocs.modeling.EpochDots()])
  # calculating the loss
  loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

  weights = model.get_weights()
  fpkl = open('drive/My Drive/pickle_models/model-' + str(carrier) + '-weights.pkl', 'wb')
  pickle.dump(weights, fpkl, protocol=pickle.HIGHEST_PROTOCOL)

  print("Testing set Mean Abs Error: {:5.2f} minutes".format(mae))
  errors = open('drive/My Drive/errors/errors.txt', mode='a')
  errors.write(str(carrier) + ':' + str(mae))
  model.save('models/model-' + str(carrier) + '.h5')
  print('OK ' + str(carrier))





Epoch: 0, loss:103.9297,  mae:7.4930,  mse:103.9296,  val_loss:98.3167,  val_mae:7.2643,  val_mse:98.3167,  
16245/16245 - 0s - loss: 100.4079 - mae: 7.2785 - mse: 100.4079
Testing set Mean Abs Error:  7.28 minutes
OK UA

Epoch: 0, loss:70.1730,  mae:6.1054,  mse:70.1729,  val_loss:63.2326,  val_mae:5.6943,  val_mse:63.2326,  
5386/5386 - 0s - loss: 61.9396 - mae: 5.6330 - mse: 61.9396
Testing set Mean Abs Error:  5.63 minutes
OK AA

Epoch: 0, loss:63.8610,  mae:5.5593,  mse:63.8609,  val_loss:50.9450,  val_mae:5.0280,  val_mse:50.9450,  
1440/1440 - 0s - loss: 65.8826 - mae: 5.3922 - mse: 65.8826
Testing set Mean Abs Error:  5.39 minutes
OK US

Epoch: 0, loss:94.8740,  mae:7.1326,  mse:94.8740,  val_loss:70.2443,  val_mae:6.1412,  val_mse:70.2443,  
973/973 - 0s - loss: 67.7627 - mae: 6.0860 - mse: 67.7627
Testing set Mean Abs Error:  6.09 minutes
OK F9

Epoch: 0, loss:75.7409,  mae:6.1963,  mse:75.7409,  val_loss:65.6162,  val_mae:5.7482,  val_mse:65.6162,  
2675/2675 - 0s - loss: 6

In [0]:
for carrier in carriers:
  # create a model and save it for each carrier
  df = pd.read_csv('drive/My Drive/carriers/carrier' + str(carrier) + 'data.csv')
  df.drop(['Unnamed: 0'], axis=1, inplace=True)

  # encode the origin 
  encoder = LabelEncoder()
  encoder.fit(df['ORIGIN_AIRPORT'])
  encoded_data_map = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
  df['ORIGIN_AIRPORT'] = encoder.fit_transform(df['ORIGIN_AIRPORT'])

  # create the train and test dataset
  train_dataset = df.sample(frac=0.8,random_state=0)
  test_dataset = df.drop(train_dataset.index)

  # getting the stats
  train_stats = train_dataset.describe()
  train_stats.pop("DEPARTURE_DELAY")
  train_stats = train_stats.transpose()

  # defining the train and test labels
  train_labels = train_dataset.pop('DEPARTURE_DELAY')
  test_labels = test_dataset.pop('DEPARTURE_DELAY')

  # normalize the data
  normed_train_data = norm(train_dataset)
  normed_test_data = norm(test_dataset)

  # define the model
  model = build_model(train_dataset)


  # train the model
  EPOCHS = 100
  # The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

  early_history = model.fit(normed_train_data, train_labels, 
                      epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                      callbacks=[early_stop, tfdocs.modeling.EpochDots()])
  # calculating the loss
  loss, mae, mse, acc = model.evaluate(normed_test_data, test_labels, verbose=2)

  weights = model.get_weights()
  fpkl = open('drive/My Drive/pickle_models/model-' + str(carrier) + '-weights.pkl', 'wb')
  pickle.dump(weights, fpkl, protocol=pickle.HIGHEST_PROTOCOL)

  print("Testing set Mean Abs Error: {:5.2f} minutes".format(mae))
  model.save('models/model-' + str(carrier) + '.h5')
  print('OK ' + str(carrier))



Epoch: 0, accuracy:0.0285,  loss:58.3596,  mae:5.3021,  mse:58.3596,  val_accuracy:0.0303,  val_loss:50.6723,  val_mae:4.9660,  val_mse:50.6723,  
16245/16245 - 0s - loss: 49.4905 - mae: 4.9166 - mse: 49.4905 - accuracy: 0.0307
Testing set Mean Abs Error:  4.92 minutes
OK UA

Epoch: 0, accuracy:0.0106,  loss:34.4828,  mae:3.8468,  mse:34.4827,  val_accuracy:0.0097,  val_loss:26.7904,  val_mae:3.5466,  val_mse:26.7903,  
.......................

KeyboardInterrupt: ignored

In [0]:
# let's create the input pipeline
from datetime import datetime

def conv_to_datetime(str_):
    return datetime.strptime(str_, '%Y-%m-%d %H:%M:%S')

def conv_to_time(str_):
    return datetime.strptime(str_, '%H:%M:%S')

In [0]:
import datetime

def string_to_time(time_string):
    if pd.isnull(time_string):
        return np.nan
    else:
        if time_string == 2400:
            time_string  = 0
        time_string = "{0:04d}".format(int(time_string))
        time_ = datetime.time(int(time_string[0:2]), int(time_string[2:4]))
        return time_
def func(x):
    return x.hour * 3600 + x.minute * 60 + x.second

In [0]:
dayOfWeek = 4
airline = 'AA'
origin = 'LAX'
dest = 'PBI'
sd = 2
ddelay = -8
sa = 750
dist = 2330

In [0]:
global train_stats

def processInput(input_):
    global train_stats
    processed = []
    time_sd = string_to_time(np.int64(input_["sd"]))
    time_sa = string_to_time(np.int64(input_["sa"]))
    time_sd = func(time_sd)
    time_sa = func(time_sa)
    # encode airlines to their numbers
    df = pd.read_csv('drive/My Drive/carriers/carrier' + str(input_["carrier"]) + 'data.csv')
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    encoder = LabelEncoder()
    encoder.fit(df['ORIGIN_AIRPORT'])
    encoded_data_map = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))


    carrier = input_["carrier"]
    
    for carr_ in carriers:
      # create a model and save it for each carrier
      if carr_ == carrier:
        df = pd.read_csv('drive/My Drive/carriers/carrier' + str(carr_) + 'data.csv')
        df.drop(['Unnamed: 0'], axis=1, inplace=True)

        # encode the origin 
        encoder = LabelEncoder()
        encoder.fit(df['ORIGIN_AIRPORT'])
        encoded_data_map = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
        print(encoded_data_map)
        df['ORIGIN_AIRPORT'] = encoder.fit_transform(df['ORIGIN_AIRPORT'])

        # create the train and test dataset
        train_dataset = df.sample(frac=0.8,random_state=0)
        test_dataset = df.drop(train_dataset.index)

        # getting the stats
        train_stats = train_dataset.describe()
        train_stats.pop("ARRIVAL_DELAY")
        train_stats = train_stats.transpose()

        # defining the train and test labels
        train_labels = train_dataset.pop('ARRIVAL_DELAY')
        test_labels = test_dataset.pop('ARRIVAL_DELAY')

        # normalize the data
        normed_train_data = norm(train_dataset)
        normed_test_data = norm(test_dataset)

        # define the model
        model = build_model(train_dataset)


        # train the model
        EPOCHS = 100
        # The patience parameter is the amount of epochs to check for improvement
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

        early_history = model.fit(normed_train_data, train_labels, 
                            epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                            callbacks=[early_stop, tfdocs.modeling.EpochDots()])
        # calculating the loss
        loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

        print("Testing set Mean Abs Error: {:5.2f} minutes".format(mae))
        # model.save('models/model-' + str(carrier) + '.h5')

        weights = model.get_weights()
        fpkl = open('model-' + str(carrier) + '-weights.pkl', 'wb')
        pickle.dump(weights, fpkl, protocol=pickle.HIGHEST_PROTOCOL)
        print('OK ' + str(carrier))

    origin = input_["origin"]
    ddelay = input_["ddelay"]
    origin_ = encoded_data_map[origin]
    dist = input_["dist"]
    weekday = input_["dayOfWeek"]
    input_ = {"time_insec_dep" : time_sd, "time_insec_arr": time_sa,
              "ORIGIN_AIRPORT": origin_, "DEPARTURE_DELAY": ddelay,
              "DISTANCE": dist, "weekday": weekday }
    
    df = pd.DataFrame([input_])
    df = norm(df)

    model = keras.models.load_model('drive/My Drive/models/model-' + str(carrier) +'.h5')
    print("OK")
    return df, model

input_ = {
          "dayOfWeek": dayOfWeek,
          "carrier": airline, 
          "origin": origin,
          "sd": sd, 
          "ddelay": ddelay,
          "sa": sa,
          "dist": dist
         }
test_input, model = processInput(input_)

{'10140': 0, '10257': 1, '10299': 2, '10397': 3, '10423': 4, '10529': 5, '10693': 6, '10713': 7, '10721': 8, '10792': 9, '10821': 10, '10994': 11, '11042': 12, '11057': 13, '11066': 14, '11109': 15, '11267': 16, '11278': 17, '11292': 18, '11298': 19, '11423': 20, '11433': 21, '11503': 22, '11540': 23, '11618': 24, '11638': 25, '11697': 26, '11884': 27, '11995': 28, '12173': 29, '12264': 30, '12266': 31, '12323': 32, '12339': 33, '12441': 34, '12451': 35, '12478': 36, '12758': 37, '12889': 38, '12892': 39, '12896': 40, '12953': 41, '12982': 42, '13198': 43, '13204': 44, '13230': 45, '13244': 46, '13256': 47, '13303': 48, '13342': 49, '13487': 50, '13495': 51, '13796': 52, '13830': 53, '13851': 54, '13871': 55, '13891': 56, '13930': 57, '13931': 58, '14027': 59, '14057': 60, '14100': 61, '14107': 62, '14122': 63, '14262': 64, '14307': 65, '14321': 66, '14492': 67, '14524': 68, '14570': 69, '14576': 70, '14635': 71, '14679': 72, '14683': 73, '14730': 74, '14747': 75, '14771': 76, '14831':

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [0]:
test_predictions_input = model.predict(test_input).flatten()
test_predictions_input

array([-10.717558], dtype=float32)

In [0]:
print(test_predictions_input[0])

-10.717558
